# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 12: Recommendation System with ALS** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Equipo**: JJAE

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ALS-Recommendation-JJAE") \
    .master("spark://80d04dce9402:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 21:02:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load the movie ratings dataset

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType

ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", LongType(), True)
])

ratings_df = spark.read.csv(
    "/home/jovyan/notebooks/data/sample_movielens_ratings.txt",
    sep="::",
    schema=ratings_schema
)

ratings_df.show(5)
ratings_df.printSchema()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
+------+-------+------+----------+
only showing top 5 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



### Train ALS Recommendation Model

In [4]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    maxIter=10,
    regParam=0.1,
    rank=5,
    coldStartStrategy="drop"
)

model = als.fit(ratings_df)

### Generate Recommendations

In [5]:
user_recommendations = model.recommendForAllUsers(numItems=5)
user_recommendations.show(truncate=False)

+------+-------------------------------------------------------------------------------------+
|userId|recommendations                                                                      |
+------+-------------------------------------------------------------------------------------+
|0     |[{92, 2.5840385}, {2, 2.316802}, {62, 2.2325232}, {25, 2.157748}, {93, 2.1528697}]   |
|10    |[{92, 2.768342}, {2, 2.6728113}, {93, 2.6242015}, {25, 2.5927775}, {49, 2.5867324}]  |
|20    |[{22, 3.5597918}, {68, 3.1278815}, {94, 3.084497}, {51, 3.0827737}, {77, 3.0246763}] |
|1     |[{22, 2.9029422}, {68, 2.630123}, {77, 2.5238972}, {62, 2.501064}, {90, 2.4797387}]  |
|11    |[{32, 5.082464}, {18, 4.705235}, {30, 4.6826043}, {27, 4.5120797}, {8, 4.229401}]    |
|21    |[{29, 4.320379}, {52, 4.2401457}, {76, 3.716108}, {63, 3.5063725}, {53, 3.4859684}]  |
|22    |[{51, 4.458179}, {75, 4.418395}, {22, 4.118836}, {74, 4.1007586}, {88, 4.0829244}]   |
|2     |[{93, 4.2531066}, {83, 4.1469526}, {8, 4.0

### Evaluate Model with RMSE

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(ratings_df)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE): {rmse:.4f}")

Root-mean-square error (RMSE): 0.5691


In [ ]:
sc.stop()